## Eryk and Jagandeep's Dataset Assembly for Module 1 Project

Please fill out:
* Student name:  **Eryk Wdowiak** and **Jagandeep Singh**
* Student pace:  full-time
* Scheduled project review date/time:  12th of Never at 13 o'clock
* Instructor name:  Fangfang Lee
* Blog post URL:  https://www.wdowiak.me/gotta-blog?answer=Yes


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
#from zodbpickle import pickle
import pickle

In [2]:
##  data files
!ls zippedData/

bom.movie_gross.csv.gz	      imdb.title.ratings.csv.gz
imdb.name.basics.csv.gz       rt.movie_info.tsv.gz
imdb.title.akas.csv.gz	      rt.reviews.tsv.gz
imdb.title.basics.csv.gz      tmdb.movies.csv.gz
imdb.title.crew.csv.gz	      tn.movie_budgets.csv.gz
imdb.title.principals.csv.gz  us-bea_gdp-deflator.csv.gz


###  load movie titles and identifiers

In [3]:
##  first load the movie titles and identifiers
imdb_title_basics = pd.read_csv('zippedData/imdb.title.basics.csv.gz',compression='gzip')
#print(imdb_title_basics.shape)

In [4]:
#imdb_title_basics.head()

In [5]:
#imdb_title_basics.loc[(imdb_title_basics['tconst']=='tt9916170')|(imdb_title_basics['tconst']=='tt9915436')|(imdb_title_basics['tconst']=='tt5820812')]
#imdb_title_basics.loc[(imdb_title_basics['primary_title']=='Titanic')]

In [6]:
##  create two dictionaries -- one of titles and one of IDs
title_dict = {}
ttlid_dict = {}

##  loop through to create the keys
for i in range(imdb_title_basics.shape[0]):
    ##  get the values
    tconst = imdb_title_basics['tconst'].loc[i]
    primary_title = imdb_title_basics['primary_title'].loc[i]
    
    ##  set up the dictionaries
    title_dict[primary_title] = { 'primary_title' : primary_title }    
    ttlid_dict[tconst] = primary_title 
    
##  now build the title dictionary
for i in range(imdb_title_basics.shape[0]):
    ##  get the values
    tconst        = imdb_title_basics['tconst'].loc[i]
    primary_title = imdb_title_basics['primary_title'].loc[i]
    start_year    = imdb_title_basics['start_year'].loc[i]
    genres        = imdb_title_basics['genres'].loc[i]

    ##  append tconst to title_dict
    try:
        title_dict[primary_title]['tconst'].append(tconst)
    except:
        title_dict[primary_title]['tconst'] = [tconst]

    ##  append start_year to title_dict
    try:
        title_dict[primary_title]['start_year'].append(start_year)
    except:
        title_dict[primary_title]['start_year'] = [start_year]

    ##  append genres to title_dict
    try:
        title_dict[primary_title]['genres'].append(genres)
    except:
        title_dict[primary_title]['genres'] = [genres]

In [7]:
# display(title_dict['Titanic'])
# display(ttlid_dict['tt2495766'])
# display(ttlid_dict['tt8852130'])

In [8]:
# display(title_dict['The Rehearsal'])
# display(ttlid_dict['tt5820812'])
# display(ttlid_dict['tt9916170'])

###  load movie cost and revenue data

In [9]:
##  get GDP deflator
gdp_deflator_data = pd.read_csv('zippedData/us-bea_gdp-deflator.csv.gz',
                                compression='gzip',header=4,nrows=2)

gdp_deflator_data = pd.DataFrame(gdp_deflator_data.iloc[1].transpose()[2:])
gdp_deflator_data.rename(columns={0: "year", 1: "gdp_deflator"},inplace=True);

##  make dictionary
deflate_dict = {}
for yr in range(1929,2020):
    deflate_dict[str(yr)] = gdp_deflator_data.loc[str(yr)][0] / 100

##  for 2020, just use 2019 value
deflate_dict['2020'] = deflate_dict['2019']

##  for years up to 1928, just use 1929 value
for yr in range(1900,1929):
    deflate_dict[str(yr)] = deflate_dict['1929']

In [10]:
#deflate_dict['1975']

In [11]:
##  next load the movie cost and revenue data
tn_movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz',compression='gzip')
#print(tn_movie_budgets.shape)
#tn_movie_budgets.head()
#list(tn_movie_budgets.columns)

In [12]:
##  define some conversion functions

##  string of dollars to float
def dollars_to_float(inlist):
    return list(map(lambda x: float(x.replace("$","").replace(",","")), inlist))

##  date string to integer year
def date_to_yrint(datestr):
    return list(map(lambda x: int(x.split()[-1]), datestr))

In [13]:
##  convert strings to float
tn_movie_budgets['production_budget'] = dollars_to_float(tn_movie_budgets['production_budget'])
tn_movie_budgets['domestic_gross'] = dollars_to_float(tn_movie_budgets['domestic_gross'])
tn_movie_budgets['worldwide_gross'] = dollars_to_float(tn_movie_budgets['worldwide_gross'])

##  divide by one million
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'] / 1000000
tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'] / 1000000
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'] / 1000000

##  calculate profit
tn_movie_budgets['profit'] = tn_movie_budgets['worldwide_gross'] - tn_movie_budgets['production_budget']

##  convert release year to integer
tn_movie_budgets['release_year'] = date_to_yrint(tn_movie_budgets['release_date'])

##  convert release date to date
tn_movie_budgets['release_date'] = tn_movie_budgets['release_date'].astype('datetime64[ns]') 
tn_movie_budgets['release_month'] = tn_movie_budgets['release_date'].dt.month

In [14]:
##  adjust for inflation
budget = []
domestic = []
worldwide = []
profit = []

for i in range(tn_movie_budgets.shape[0]):
    ##  get year and its deflator
    year = tn_movie_budgets['release_year'].iloc[i]
    deflator = deflate_dict[str(year)]

    ##  append to the lists
    budget.append(tn_movie_budgets['production_budget'].iloc[i] / deflator)
    domestic.append(tn_movie_budgets['domestic_gross'].iloc[i] / deflator)
    worldwide.append(tn_movie_budgets['worldwide_gross'].iloc[i] / deflator)
    profit.append(tn_movie_budgets['profit'].iloc[i] / deflator)

##  replace them in the dataframe
tn_movie_budgets['production_budget'] = budget
tn_movie_budgets['domestic_gross'] = domestic
tn_movie_budgets['worldwide_gross'] = worldwide
tn_movie_budgets['profit'] = profit

##  take logs
tn_movie_budgets['ln_budget'] = np.log(tn_movie_budgets['production_budget'])
tn_movie_budgets['ln_domestic'] = np.log(tn_movie_budgets['domestic_gross'])
tn_movie_budgets['ln_worldwide'] = np.log(tn_movie_budgets['worldwide_gross'])

del budget, domestic, worldwide, profit

/usr/lib/python3/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/usr/lib/python3/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


In [15]:
##  print some information
#tn_movie_budgets.columns
#tn_movie_budgets.info()
tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit,release_year,release_month,ln_budget,ln_domestic,ln_worldwide
0,1,2009-12-18,Avatar,447.349585,800.500637,2922.345669,2474.996083,2009,12,6.103340,6.685237,7.980142
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,418.475713,245.687718,1065.720739,647.245026,2011,5,6.036619,5.504061,6.971407
2,3,2019-06-07,Dark Phoenix,311.540344,38.063421,133.305755,-178.234590,2019,6,5.741529,3.639254,4.892645
3,4,2015-05-01,Avengers: Age of Ultron,315.708051,438.329849,1339.814894,1024.106843,2015,5,5.754818,6.082972,7.200287
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,294.079448,575.339427,1221.516733,927.437285,2017,12,5.683850,6.354960,7.107849


In [16]:
# tn_movie_budgets.loc[(tn_movie_budgets['movie']=='King Kong')]

In [17]:
# tn_movie_budgets.loc[tn_movie_budgets.movie == 'Titanic']

In [18]:
##  create some histograms
#tn_movie_budgets.release_year.hist();
#tn_movie_budgets.profit.hist();

In [19]:
##  now create profit dictionary
profit_dict = {}

for i in range(tn_movie_budgets.shape[0]):
    ##  get the values
    movie_title   = tn_movie_budgets['movie'].loc[i]
    release_year  = tn_movie_budgets['release_year'].loc[i]
    release_month = tn_movie_budgets['release_month'].loc[i]
    release_date  = tn_movie_budgets['release_date'].loc[i]
    profit_dict[movie_title] = {
        'movie_title'   : movie_title,
        'release_year'  : release_year,
        'release_month' : release_month,
        'release_date'  : release_date
    }
    
for i in range(tn_movie_budgets.shape[0]):
    ##  get the values again
    movie_title  = tn_movie_budgets['movie'].loc[i]
    release_year = tn_movie_budgets['release_year'].loc[i]
    release_month = tn_movie_budgets['release_month'].loc[i]
    release_date = tn_movie_budgets['release_date'].loc[i]

    ##  let's assume that most recent movie is the movie in the IMDB
    if (release_year >= profit_dict[movie_title]['release_year']):
        ##  add them to the dictionary
        profit_dict[movie_title] = {
            'movie_title'   : movie_title,
            'release_year'  : release_year,
            'release_month' : release_month,
            'release_date'  : release_date,
            'production_budget' : tn_movie_budgets['production_budget'].loc[i],
            'domestic_gross' : tn_movie_budgets['domestic_gross'].loc[i],
            'worldwide_gross' : tn_movie_budgets['worldwide_gross'].loc[i],
            'profit' : tn_movie_budgets['profit'].loc[i],
            'ln_budget' : tn_movie_budgets['ln_budget'].loc[i],
            'ln_domestic' : tn_movie_budgets['ln_domestic'].loc[i],
            'ln_worldwide' : tn_movie_budgets['ln_worldwide'].loc[i]
    }

In [20]:
# profit_dict['Titanic']

###  get the principals

In [21]:
imdb_title_principals = pd.read_csv('zippedData/imdb.title.principals.csv.gz',compression='gzip')
# print(imdb_title_principals.shape)
# print('number of unique titles: ' + str(len(list(imdb_title_principals.tconst.unique()))))
# print('unique categories:')
# print(list(imdb_title_principals.category.unique()))
# imdb_title_principals.head(10)
#list(imdb_title_principals.columns)

In [22]:
##  let's create a dictionary of principals
principals_dict = {}

##  loop through to create keys and build the dictionary
for i in range(imdb_title_principals.shape[0]):
    ##  get the values
    tconst = imdb_title_principals['tconst'].loc[i]
    nconst = imdb_title_principals['nconst'].loc[i]
    category = imdb_title_principals['category'].loc[i]

    ##  create the keys
    try:
        principals_dict[tconst]['tconst'] = tconst
    except:
        principals_dict[tconst] = {'tconst' : tconst}
    
    ##  append nconst to the appropriate role
    if (category in ['producer','director','actor','actress']):
        try:
            principals_dict[tconst][category].append(nconst)
        except:
            principals_dict[tconst][category] = [nconst]

In [23]:
# principals_dict['tt0323808']

###  get the ratings

In [24]:
##  now let's include the ratings
imdb_title_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz',compression='gzip')
# print(imdb_title_ratings.shape)
# print('number of unique titles: ' + str(len(list(imdb_title_ratings.tconst.unique()))))
# imdb_title_ratings.head()
#list(imdb_title_ratings.columns)

In [25]:
##  let's create a dictionary of ratings
ratings_dict = {}

##  loop through to build the dictionary
for i in range(imdb_title_ratings.shape[0]):
    ##  get the values
    tconst = imdb_title_ratings['tconst'].loc[i]
    rating = imdb_title_ratings['averagerating'].loc[i]
    nuvote = imdb_title_ratings['numvotes'].loc[i]

    ##  create the keys
    ratings_dict[tconst] = {
        'tconst' : tconst,
        'rating' : rating,
        'nuvote' : nuvote
    }
    

In [26]:
# ratings_dict['tt1043726']


###  now merge all those dictionaries

In [27]:
##  now find titles in "profit_dict" that match titles in "title_dict"
profit_keys = profit_dict.keys()
title_keys  = title_dict.keys()

##  those titles are at the intersection
profit_titles = list(set(profit_keys) & set(title_keys))

In [28]:
##  columns for merged dataframe
tconst = []
movie_title = []
genres = []
release_year = []
release_month = []
release_date = []
production_budget = []
domestic_gross = []
worldwide_gross = []
profit = []
ln_budget = []
ln_domestic = []
ln_worldwide = []

##  and the principals 
actresses = []
actors = []
directors = []
producers = []

##  and the ratings
ratings = []
nuvotes = []

for movie in profit_titles:
    tconst.append(title_dict[movie]['tconst'])
    movie_title.append(profit_dict[movie]['movie_title'])
    ##  we'll pick up genres below
    release_year.append(profit_dict[movie]['release_year'])
    release_month.append(profit_dict[movie]['release_month'])
    release_date.append(profit_dict[movie]['release_date'])
    production_budget.append(profit_dict[movie]['production_budget'])
    domestic_gross.append(profit_dict[movie]['domestic_gross'])
    worldwide_gross.append(profit_dict[movie]['worldwide_gross'])
    profit.append(profit_dict[movie]['profit'])
    ln_budget.append(profit_dict[movie]['ln_budget'])
    ln_domestic.append(profit_dict[movie]['ln_domestic'])
    ln_worldwide.append(profit_dict[movie]['ln_worldwide'])
    
    
    ##  movie principals 
    mv_actresses = []
    mv_actors = []
    mv_directors = []
    mv_producers = []
    
    ##  movie genres
    mv_genres = []
    
    ##  movie ratings
    mv_ratings = []
    mv_nuvotes = []

    for ttlid in title_dict[movie]['tconst']:
        try:
            mv_actresses.append(principals_dict[ttlid]['actress'])
        except:
            blah = 'blah'
    
        try:
            mv_actors.append(principals_dict[ttlid]['actor'])
        except:
            blah = 'blah'
        
        try:
            mv_directors.append(principals_dict[ttlid]['director'])
        except:
            blah = 'blah'
            
        try:
            mv_producers.append(principals_dict[ttlid]['producer'])
        except:
            blah = 'blah'
            
        try:
            mv_ratings.append(ratings_dict[ttlid]['rating'])
        except:
            blah = 'blah'
            
        try:
            mv_nuvotes.append(ratings_dict[ttlid]['nuvote'])
        except:
            blah = 'blah'
            
        try:
            comma = ','
            genre_str = comma.join(title_dict[ttlid_dict[ttlid]]['genres'])           
            mv_genres.append(genre_str)
        except:
            blah = 'blah'


    ##  append principals
    actresses.append(mv_actresses)
    actors.append(mv_actors)
    directors.append(mv_directors)
    producers.append(mv_producers)

    ##  make genres one long string, then split it and make list unique
    comma = ','
    mv_genres = comma.join(mv_genres)
    mv_genres = mv_genres.split(comma)
    genres.append(list(set(mv_genres)))
    
    ##  just return rating with most votes
    try:
        which_max = mv_nuvotes.index(max(mv_nuvotes))
        ratings.append(mv_ratings[which_max])
        nuvotes.append(mv_nuvotes[which_max])
    except:
        ratings.append(None)
        nuvotes.append(None)

##  now create a merged dataframe
movie_df = pd.DataFrame({'tconst' : tconst,
                         'movie_title' : movie_title, 
                         'genres' : genres,
                         'release_year' : release_year,
                         'release_month' : release_month,
                         'release_date' : release_date,
                         'production_budget' : production_budget, 
                         'domestic_gross' : domestic_gross,
                         'worldwide_gross' : worldwide_gross,
                         'profit' : profit,
                         'ln_budget' : ln_budget, 
                         'ln_domestic' : ln_domestic,
                         'ln_worldwide' : ln_worldwide,                        
                         'actresses' : actresses, 
                         'actors' : actors, 
                         'directors' : directors, 
                         'producers' : producers, 
                         'ratings' : ratings, 
                         'nuvotes' : nuvotes
                        })

del tconst, movie_title, genres, release_year, release_month, release_date
del production_budget, domestic_gross, worldwide_gross, profit
del ln_budget, ln_domestic, ln_worldwide
del actresses, actors, directors, producers, ratings, nuvotes

In [29]:
#movie_df.info()
# movie_df.shape

In [30]:
# movie_df.head()

###  get their names

In [31]:
##  and let's get their real names!
imdb_name_basics = pd.read_csv('zippedData/imdb.name.basics.csv.gz',compression='gzip')
# print(imdb_name_basics.shape)
# print('number of unique nconst: ' + str(len(list(imdb_name_basics.nconst.unique()))))
# imdb_name_basics.head()
#list(imdb_name_basics.columns)

In [32]:
##  let's create a dictionary of ratings
names_dict = {}

##  loop through to build the dictionary
for i in range(imdb_title_ratings.shape[0]):
    ##  get the values
    nconst = imdb_name_basics['nconst'].loc[i]
    prname = imdb_name_basics['primary_name'].loc[i]

    ##  create the keys
    names_dict[nconst] = prname
    

###  let's save this monster

In [34]:
pickle.dump(movie_df,open("movie-df_2020-07-09a.p","wb"))

#### xxx

##  stuff we're not gonna use

In [ ]:
##  create new dataframe for statistical analysis

##  first set the indices to the movie's title
#imdb_title_basics.set_index('primary_title',inplace=True)
#tn_movie_budgets.set_index('movie',inplace=True)

##  now create the new dataframe
#bad_df = tn_movie_budgets.join(imdb_title_basics,how='inner')

In [ ]:
#imdb_title_crew = pd.read_csv('zippedData/imdb.title.crew.csv.gz',compression='gzip')
#print(imdb_title_crew.shape)
#print('number of unique titles: ' + str(len(list(imdb_title_crew.tconst.unique()))))
#imdb_title_crew.head()
#list(imdb_title_crew.columns)

In [ ]:
#imdb_title_basics.head()
#tn_movie_budgets.head()
#bad_df.sort_values(by='profit',ascending=False).head(10)

In [ ]:
#imdb_title_basics.shape
#tn_movie_budgets.shape
#bad_df.shape
#bad_df.loc[(bad_df['original_title']=='Titanic')]

In [ ]:
#bom_movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz',compression='gzip')
#bom_movie_gross.head()
#list(bom_movie_gross.columns)

In [ ]:
#imdb_title_akas = pd.read_csv('zippedData/imdb.title.akas.csv.gz',compression='gzip')
#print(imdb_title_akas.shape)
#imdb_title_akas.head()
#list(imdb_title_akas.columns)
#imdb_title_akas.sort_values(by='title_id',ascending=False).head(10)
#imdb_title_akas.loc[imdb_title_akas['title'] == "The Rehearsal"] 
#imdb_title_akas.loc[(imdb_title_akas['title_id']=='tt9916170')|(imdb_title_akas['title_id']=='tt9915436')|(imdb_title_akas['title_id']=='tt5820812')]

In [ ]:
#rt_movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz',compression='gzip',sep='\t')
#rt_movie_info.head()
#list(rt_movie_info.columns)

In [ ]:
#rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz',compression='gzip',sep='\t',encoding='unicode_escape')
#rt_reviews.head()
#list(rt_reviews.columns)

In [ ]:
#tmdb_movies = pd.read_csv('zippedData/tmdb.movies.csv.gz',compression='gzip')
#tmdb_movies.head()
#list(tmdb_movies.columns)